# Multi Volume functions

Routines to match multiple polygons in a contour slice with polygons in the next slice

- Measure overlap area 
- Identify some overlap
- Identify most overlap



## Setup

### Imports

In [1]:
# Type imports
from typing import Any, Dict, Tuple

# Standard Libraries
from pathlib import Path
from itertools import combinations
from math import sqrt, pi
import re
from statistics import mean
from pprint import pprint

# Shared Packages
import numpy as np
import pandas as pd
import xlwings as xw
import PySimpleGUI as sg
import pydicom
from shapely.geometry import Polygon
from shapely import points
import shapely

import RS_DICOM_Utilities


## File Paths

In [2]:
base_path = Path.cwd()
data_path = base_path / 'Test Data'
dicom_path = data_path / 'StructureVolumeTests' / 'MultiVolume_A'


Test Data\StructureVolumeTests\MultiVolume_A\RS.GJS_Struct_Tests.MultiVolume_A.dcm

In [3]:
multi_volume_a_file = dicom_path / 'RS.GJS_Struct_Tests.MultiVolume_A.dcm'
structure_set_info = RS_DICOM_Utilities.get_structure_file_info(multi_volume_a_file)
structure_set_info

{'PatientName': 'StructureVolumes^Test',
 'PatientLastName': 'StructureVolumes',
 'PatientID': 'GJS_Struct_Tests',
 'StructureSet': 'MultiVolume_A',
 'StudyID': 'Phantom2',
 'SeriesNumber': '9',
 'File': WindowsPath("d:/OneDrive - Queen's University/Python/Projects/StructureRelations/Test Data/StructureVolumeTests/MultiVolume_A/RS.GJS_Struct_Tests.MultiVolume_A.dcm")}

### Relevant Info
#### Contour Stats

In [4]:
dataset = pydicom.dcmread(structure_set_info['File'])
name_lookup = RS_DICOM_Utilities.get_names_nums(dataset)

contour_sets = RS_DICOM_Utilities.read_contours(dataset)
contour_stats = pd.DataFrame(cntr.info for cntr in contour_sets.values())


#### ROI Number Lookup

In [5]:
roi_lookup = {cntr.structure_id: cntr.roi_num
              for cntr in contour_sets.values()}
roi_lookup

{'BODY': '1',
 'AdjacentSpheres': '7',
 'AdjacentShells': '8',
 'SingleVolume': '9'}

In [6]:
contour_slices = RS_DICOM_Utilities.build_slice_table(contour_sets)


In [7]:
#xw.view(contour_slices)
#xw.view(contour_slices.describe())


In [8]:
structure_id = 'AdjacentSpheres'


In [9]:
RS_DICOM_Utilities.has_gaps(structure_id, contour_slices)

False

In [15]:
roi = roi_lookup[structure_id]
structure = contour_sets[roi]
structure.info

{'ROI Num': '7',
 'StructureID': 'AdjacentSpheres',
 'Sup Slice': 9.4,
 'Inf Slice': 1.1,
 'Length': 8.3,
 'Thickness': 0.099,
 'Volume': 131.034,
 'Eq Sp Diam': 6.302,
 'Center of Mass': (4.435, -2.841, 5.226),
 'Resolution': 10.87,
 'Colour': ('255', '255', '0')}

# Done to Here


## Routine to match multiple polygons in a contour slice with polygons in the next slice
- Measure overlap area 
- Identify some overlap
- Identify most overlap



**What format should this information be in??**

In [18]:
regions = pd.Series({slice: contour.region_count
                     for slice, contour in structure.contours.items()})

In [19]:
regions.describe()

count    84.000000
mean      1.190476
std       0.395035
min       1.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       2.000000
dtype: float64

In [14]:

slice_index = structure.neighbours
for ref in slice_index.itertuples():
    this_slice = structure.contours[ref.slice]
    if this_slice.region_count > 1:
        if ref.inf:
            next_slice = structure.contours[ref.inf]


In [19]:
slice = 5
ref = slice_index.T[slice]
this_slice = contour_sets[roi].contours[ref.slice]
next_slice = contour_sets[roi].contours[ref.inf]

## Measure overlap area 

In [21]:

match = {}
for idx1, poly1 in enumerate(this_slice.contour.geoms):
    sub_match = {}
    for idx2, poly2 in enumerate(next_slice.contour.geoms):
        area = shapely.intersection(poly1, poly2).area
        sub_match[(ref.inf, idx2)] = area
    match[(slice, idx1)] = sub_match
pd.DataFrame(match)

5          
               0         1
5.1 0  12.092769  0.000000
    1   0.000000  8.267374

# Look for Projection overlap
intersection between projection of centre-of-mass of contour from neighbouring slice onto contour in current slice
- If no overlap, do not interpolate
- This is also important where there is more than one contour per slice. 
- Need to to this separately for each contour polygon in the slice

**Overlaps ignores _Z_ component of contour**


In [ ]:
a = contour_sets[roi_lookup['continuous']].contours[-9.9].contour
b = contour_sets[roi_lookup['continuous']].contours[-9.8].contour
a.overlaps(b)

In [ ]:
c = contour_sets[roi_lookup['missing3rd']].contours[-9.9].contour
a.overlaps(c)

In [ ]:
structure_id = 'missing3rd'

inf = contour_slices[structure_id].dropna().index.min()
sup = contour_slices[structure_id].dropna().index.max()

contour_range = (contour_slices.index <= sup) & (contour_slices.index >= inf)
structure_slices = contour_slices.loc[contour_range, structure_id]
missing_slices = structure_slices.isna()


In [ ]:
structure_id = 'rndOffsetShell'

inf = contour_slices[structure_id].dropna().index.min()
sup = contour_slices[structure_id].dropna().index.max()

contour_range = (contour_slices.index <= sup) & (contour_slices.index >= inf)
structure_slices = contour_slices.loc[contour_range, structure_id]
missing_slices = structure_slices.isna()


In [ ]:
roi = roi_lookup[structure_id]
roi

# Metrics

In [ ]:
contour_sets[2].contours[0].contour.distance(contour_sets[3].contours[0].contour)

In [ ]:
contour_sets[3].contours[0].contour.distance(contour_sets[13].contours[0].contour)

In [ ]:
contour_sets[13].contours[0].contour.bounds

In [ ]:
contour_sets[3].contours[0].contour.bounds

`bounds(geometry, **kwargs)`
Computes the bounds (extent) of a geometry.

For each geometry these 4 numbers are returned: 
> (min x, min y, max x, max y)

In [ ]:
contour_sets[2].contours[0].contour.bounds

In [ ]:
contour = contour_sets[2].contours[0].contour.bounds

In [ ]:
slices = contour_sets[3].contours[0].contour.bounds


In [ ]:
[(contour[0], contour[1]), (contour[2], contour[3]), (contour[0], contour[3]), (contour[2], contour[1])]

**shapely.prepare**

`prepare(geometry, **kwargs)`
> Prepare a geometry, improving performance of other operations.
>
> A prepared geometry is a normal geometry with added information such as an index on the line segments. This improves the performance of the following operations: contains, contains_properly, covered_by, covers, crosses, disjoint, intersects, overlaps, touches, and within.
>
> Note that if a prepared geometry is modified, the newly created Geometry object is not prepared. In that case, prepare should be called again.
>
> This function does not recompute previously prepared geometries; it is efficient to call this function on an array that partially contains prepared geometries.
>
> This function does not return any values; geometries are modified in place.


**shapely.set_precision**

`set_precision(geometry, grid_size, mode='valid_output', **kwargs)`

> Returns geometry with the precision set to a precision grid size.
>
> By default, geometries use double precision coordinates (grid_size = 0).
> 
> Coordinates will be rounded if a precision grid is less precise than the input geometry. Duplicated vertices will be dropped from lines and polygons for grid sizes greater than 0. Line and polygon geometries may collapse to empty geometries if all vertices are closer together than grid_size. Z values, if present, will not be modified.
> 
> **Note:** subsequent operations will always be performed in the precision of the geometry with higher precision (smaller “grid_size”). That same precision will be attached to the operation outputs.
> 
> **Also note:** input geometries should be geometrically valid; unexpected results may occur if input geometries are not.
> 
> *Returns* None if geometry is None.
>
> **Parameters**:
>> **geometry**: Geometry or array_like
>> 
>> **grid_size**: float
>>> Precision grid size. If 0, will use double precision (will not modify geometry if precision grid size was not previously set). If this value is more precise than input geometry, the input geometry will not be modified.
>> 
>> **mode**: {‘valid_output’, ‘pointwise’, ‘keep_collapsed’}, default ‘valid_output’
>>> This parameter determines how to handle invalid output geometries. There are three modes:
>>> *‘valid_output’* (default): The output is always valid. Collapsed geometry elements (including both polygons and lines) are removed. Duplicate vertices are removed.
>>> *‘pointwise’*: Precision reduction is performed pointwise. Output geometry may be invalid due to collapse or self-intersection. Duplicate vertices are not removed. In GEOS this option is called NO_TOPO.
>>> *‘keep_collapsed’*: Like the default mode, except that collapsed linear geometry elements are preserved. Collapsed polygonal input elements are removed. Duplicate vertices are removed.
>> 
>> **kwargs: See NumPy ufunc docs for other keyword arguments.

> **Examples**
```python
from shapely import LineString, Point
set_precision(Point(0.9, 0.9), 1.0)
<POINT (1 1)>
set_precision(Point(0.9, 0.9, 0.9), 1.0)
<POINT Z (1 1 0.9)>
set_precision(LineString([(0, 0), (0, 0.1), (0, 1), (1, 1)]), 1.0)
<LINESTRING (0 0, 0 1, 1 1)>
set_precision(LineString([(0, 0), (0, 0.1), (0.1, 0.1)]), 1.0, mode="valid_output")
<LINESTRING Z EMPTY>
set_precision(LineString([(0, 0), (0, 0.1), (0.1, 0.1)]), 1.0, mode="pointwise")
<LINESTRING (0 0, 0 0, 0 0)>
set_precision(LineString([(0, 0), (0, 0.1), (0.1, 0.1)]), 1.0, mode="keep_collapsed")
<LINESTRING (0 0, 0 0)>
set_precision(None, 1.0) is None
```